In [ ]:
# FINAL CODE - Versión Final Limpia y Completa

import time
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.ensemble import IsolationForest
from datetime import datetime, timezone
import requests
import pytz # Import pytz for timezone handling
import matplotlib.pyplot as plt # Import matplotlib for plotting
from google.colab import files # Import files for download
import sys # Import sys to exit if needed
import json # Import json for reading configuration files


# --------- CONFIGURACION (Estas variables se establecen al inicio de la ejecucion) ----------
# No es necesario modificar aqui directamente, se piden los valores al ejecutar o se cargan desde archivo.
SYMBOLS = []
PERIOD = ""
INTERVAL = ""
Z_THRESHOLD = 0.0
IF_CONTAMINATION = 0.0
# Added %K_smooth and %D to the default IF_FEATURES
IF_FEATURES = ["logret", "vol_20", "z_price", "vol_z", "price_vol_interaction", "rsi", "macd", "upper_band", "lower_band", "atr", "cmf", "%K_smooth", "%D"]
DISCORD_WEBHOOK_URL = ""
ANOMALY_THRESHOLD = 0.0
# New: Signal parameters with defaults
TREND_WINDOW = 50
RSI_BUY_THRESHOLD = 30
RSI_SELL_THRESHOLD = 70
# New: Variables for backtesting configuration
INITIAL_CAPITAL = 10000
COMMISSION_RATE = 0.001


# ----------------------------------------------------

def send_discord_message(text, webhook_url):
    """
    Envia un mensaje a un canal de Discord usando un webhook con diagnóstico.
    Si webhook_url está vacío, simplemente imprime un mensaje indicando que no se enviará.
    """
    if not webhook_url:
        print("Diagnóstico Discord: URL de Webhook no configurada. No se intentara enviar mensaje.")
        return

    data = {"content": text}
    try:
        print(f"Diagnóstico Discord: Intentando enviar mensaje a {webhook_url}...")
        response = requests.post(webhook_url, json=data)
        response.raise_for_status() # Levanta una excepcion para codigos de estado HTTP de error (4xx o 5xx)
        print("Diagnóstico Discord: Mensaje enviado exitosamente.")
    except requests.exceptions.RequestException as e:
        print(f"Diagnóstico Discord: Error al enviar mensaje a Discord - {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Diagnóstico Discord: Código de estado HTTP - {e.response.status_code}")
            print(f"Diagnóstico Discord: Cuerpo de la respuesta - {e.response.text}")
    except Exception as e:
        print(f"Diagnóstico Discord: Ocurrio un error inesperado al enviar a Discord:", e)


def fetch_data(symbol, period, interval):
    """
    Descarga los datos OHLCV de Yahoo Finance para un simbolo dado
    y aplana el MultiIndex si existe, renombrando columnas para manejo consistente.
    Incluye manejo de errores y diagnóstico si no se obtienen datos.
    """
    print(f"Descargando datos para: {symbol} (Periodo: {period}, Intervalo: {interval})")
    df = pd.DataFrame() # Inicializa un dataframe vacio

    try:
        # Use auto_adjust=True to get adjusted close price (Open, High, Low, Close)
        df = yf.download(tickers=symbol, period=period, interval=interval, progress=False, threads=False, auto_adjust=True)

        if df.empty:
            print(f"Diagnóstico Fetch: yfinance.download retornó un dataframe vacio para {symbol}. Esto puede deberse a:")
            print("- Símbolo incorrecto o no encontrado.")
            print("- Combinación de Periodo/Intervalo no soportada o sin datos disponibles.")
            print("- Problemas temporales con la fuente de datos (Yahoo Finance).")
            return pd.DataFrame() # Retorna un dataframe vacio si no hay datos

        df = df.dropna() # Eliminar filas con valores faltantes despues de la descarga

        if df.empty:
             print(f"Diagnóstico Fetch: El dataframe para {symbol} quedo vacio despues de eliminar NaNs. Puede que no haya suficientes datos limpios para el periodo/intervalo especificado.")


        # Handle MultiIndex from multiple symbols download (yfinance returns MultiIndex for >1 symbol)
        # Use the global SYMBOLS list to determine if multiple symbols were requested initially
        if isinstance(df.columns, pd.MultiIndex):
            print(f"Diagnóstico Fetch: Detectado MultiIndex para {symbol}. Aplanando columnas.")
            # Create new column names by joining levels (e.g., ('Close', 'AAPL') -> 'Close_AAPL')
            # Ensure only relevant columns ('Open', 'High', 'Low', 'Close', 'Volume') are kept and renamed

            # Select only the columns relevant to the current symbol based on the original MultiIndex structure
            # This is a safer way to select columns for a specific symbol from a multi-symbol download MultiIndex
            try:
                # Ensure the MultiIndex has the expected levels and structure
                # Check if the symbol is actually in the second level of the MultiIndex
                if len(df.columns.levels) > 1 and symbol in df.columns.levels[1]:
                    cols_to_select = [(col_type, symbol) for col_type in ['Open', 'High', 'Low', 'Close', 'Volume'] if (col_type, symbol) in df.columns]
                    if cols_to_select:
                         df_single_symbol = df[cols_to_select]
                         # Rename the columns to single level names (e.g., ('Close', 'AAPL') -> 'Close')
                         df_single_symbol.columns = [col[0] for col in df_single_symbol.columns.values]
                         df = df_single_symbol # Replace the original df with the single symbol flat df
                         print(f"Diagnóstico Fetch: Columnas aplanadas y seleccionadas para {symbol}: {list(df.columns)}")
                    else:
                         print(f"Diagnóstico Fetch: No se encontraron columnas OHLCV/Volume con MultiIndex para el simbolo {symbol}.")
                         return pd.DataFrame()
                else:
                     print(f"Diagnóstico Fetch: La estructura del MultiIndex no es la esperada para el simbolo {symbol} o el simbolo no esta en el MultiIndex.")
                     # Attempt to handle cases where yfinance might return a MultiIndex even for a single symbol in a different format
                     if len(df.columns.levels) == 1 and len(df.columns) > 0:
                          print("Diagnóstico Fetch: Intentando aplanar MultiIndex de un nivel.")
                          df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns.values]
                          print(f"Diagnóstico Fetch: Columnas aplanadas: {list(df.columns)}")
                     else:
                          print(f"Niveles del MultiIndex: {df.columns.levels}")
                          return pd.DataFrame()
            except KeyError as e:
                 print(f"Diagnóstico Fetch: Error al seleccionar columnas para el simbolo {symbol} despues de MultiIndex: {e}")
                 print(f"Columnas disponibles en el dataframe MultiIndex: {list(df.columns)}")
                 return pd.DataFrame()


        # For single symbol download, yfinance might return a MultiIndex with one level, flatten that too
        # Or simply rename the columns to standard names if not already
        elif isinstance(df.columns, pd.MultiIndex) and len(df.columns.levels) > 0 and len(df.columns.levels[0]) > 0 and len(df.columns.levels[1]) == 1:
             df.columns = [col[0] for col in df.columns.values]
             print(f"Diagnóstico Fetch: Detectado MultiIndex de un nivel para {symbol}. Aplanando columnas: {list(df.columns)}")
        # If it's already a flat index but column names are lowercase, normalize them
        elif all(col.islower() for col in df.columns):
            df.columns = [col.capitalize() for col in df.columns]
            print(f"Diagnóstico Fetch: Columnas en minusculas normalizadas para {symbol}: {list(df.columns)}")
        else:
             print(f"Diagnóstico Fetch: Columnas ya planas o no estandar para {symbol}: {list(df.columns)}")


    except Exception as e:
        print(f"Diagnóstico Fetch: Ocurrio un error inesperado durante la descarga de datos para {symbol}: {e}")
        return pd.DataFrame() # Retorna un dataframe vacio en caso de excepcion

    if df.empty:
        print(f"Diagnóstico Fetch: El dataframe final para {symbol} esta vacio despues de procesar.")

    return df

def feature_engineer(df, symbol):
    """
    Calcula caracteristicas (features) para la deteccion de anomalias,
    incluyendo indicadores técnicos comunes como RSI, MACD, Bollinger Bands, ATR, CMF, y Stochastic Oscillator.
    Asume que el dataframe tiene columnas aplanadas ('Open', 'High', 'Low', 'Close', 'Volume')
    después de fetch_data, independientemente de si fue descarga de uno o múltiples símbolos.
    """
    df = df.copy()

    # Determine the correct column names - assuming fetch_data has normalized them to single level
    close_col = "Close"
    high_col = "High"
    low_col = "Low"
    volume_col = "Volume"

    # Ensure the required columns exist before proceeding with calculations
    required_cols = [close_col, high_col, low_col, volume_col]
    if not all(col in df.columns for col in required_cols):
        print(f"Error: Missing required columns for feature engineering for {symbol}.")
        print(f"Expected: {required_cols}, Found: {list(df.columns)}")
        return pd.DataFrame() # Return empty DataFrame if essential columns are missing


    # Retornos logaritmicos: Mide el cambio porcentual en el precio en una escala logaritmica.
    df["logret"] = np.log(df[close_col] / df[close_col].shift(1))
    # Volatilidad realizada (usando una ventana mas pequeña para mayor sensibilidad)
    # Mide la dispersion de los retornos en un periodo corto (10 periodos).
    df["vol_10"] = df["logret"].rolling(window=10).std() * np.sqrt(252)
    # Volatilidad realizada anualizada (ventana mas grande)
    # Mide la dispersion de los retornos en un periodo mas largo (20 periodos), anualizada.
    df["vol_20"] = df["logret"].rolling(window=20).std() * np.sqrt(252)
    # Z-score del precio (desviacion del precio respecto a su media movil)
    # Indica cuantas desviaciones estandar esta el precio actual de su media movil de 20 periodos.
    df["z_price"] = (df[close_col] - df[close_col].rolling(20).mean()) / df[close_col].rolling(20).std()
    # Z-score del volumen (desviacion del volumen respecto a su media movil)
    # Indica cuantas desviaciones estandar esta el volumen actual de su media movil de 20 periodos.
    df["vol_z"] = (df[volume_col] - df[volume_col].rolling(20).mean()) / df[volume_col].rolling(20).std()
    # Interaccion entre precio y volumen (puede indicar movimientos inusuales)
    # Combina el cambio de precio con la desviacion del volumen para identificar eventos donde
    # grandes cambios de precio ocurren con volumen inusual.
    df["price_vol_interaction"] = df["logret"] * df["vol_z"]
    # Media movil del logret para capturar tendencias a corto plazo
    # Suaviza los retornos logaritmicos para identificar la direccion promedio del movimiento.
    df["logret_ma5"] = df["logret"].rolling(window=5).mean()

    # Calcular RSI (Relative Strength Index): Indicador de momentum que mide la velocidad y cambio de los movimientos de precios.
    # Window for RSI (commonly 14 periods)
    rsi_window = 14
    delta = df[close_col].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.ewm(com=rsi_window - 1, adjust=False).mean()
    avg_loss = loss.ewm(com=rsi_window - 1, adjust=False).mean()
    # Avoid division by zero
    rs = avg_gain / (avg_loss + 1e-10)
    df["rsi"] = 100 - (100 / (1 + rs))

    # Calcular MACD (Moving Average Convergence Divergence): Indicador de seguimiento de tendencia que muestra la relacion entre dos medias moviles del precio de un activo.
    # EMA windows for MACD (commonly 12, 26, 9 periods)
    ema_fast_window = 12
    ema_slow_window = 26
    signal_window = 9
    ema_fast = df[close_col].ewm(span=ema_fast_window, adjust=False).mean()
    ema_slow = df[close_col].ewm(span=ema_slow_window, adjust=False).mean()
    df["macd"] = ema_fast - ema_slow # Linea MACD
    df["macd_signal"] = df["macd"].ewm(span=signal_window, adjust=False).mean() # Linea de señal
    df["macd_hist"] = df["macd"] - df["macd_signal"] # Histograma MACD

    # Calcular Bollinger Bands: Indicadores de volatilidad que se basan en una media móvil simple y dos bandas de desviación estándar.
    bb_window = 20
    df['rolling_mean'] = df[close_col].rolling(window=bb_window).mean()
    df['rolling_std'] = df[close_col].rolling(window=bb_window).std()
    df['upper_band'] = df['rolling_mean'] + (df['rolling_std'] * 2)
    df['lower_band'] = df['rolling_mean'] - (df['rolling_std'] * 2)


    # Calcular Average True Range (ATR): Mide la volatilidad del mercado.
    atr_window = 14
    # Calculate True Range (TR)
    df['tr1'] = abs(df[high_col] - df[low_col])
    df['tr2'] = abs(df[high_col] - df[close_col].shift(1))
    df['tr3'] = abs(df[low_col] - df[close_col].shift(1))
    df['tr'] = df[['tr1', 'tr2', 'tr3']].max(axis=1)
    # Calculate ATR using Exponential Moving Average (EMA)
    df['atr'] = df['tr'].ewm(span=atr_window, adjust=False).mean()


    # Calcular Chaikin Money Flow (CMF): Combina precio y volumen para medir la presión de compra y venta.
    cmf_window = 20
    # Calculate Money Flow Volume (MFV)
    # Avoid division by zero
    df['mf_multiplier'] = ((df[close_col] - df[low_col]) - (df[high_col] - df[close_col])) / (df[high_col] - df[low_col] + 1e-10)

    # Ensure multiplication with the correct Volume column and assign to a single column 'mfv'
    # Check if the volume_col exists in the DataFrame before accessing
    if volume_col in df.columns:
        df['mfv'] = df['mf_multiplier'] * df[volume_col]
        # Calculate CMF
        # Ensure sum is calculated over the correct 'mfv' column
        # Avoid division by zero if the sum of volume is zero
        volume_sum_rolling = df[volume_col].rolling(window=cmf_window).sum()
        df['cmf'] = df['mfv'].rolling(window=cmf_window).sum() / (volume_sum_rolling + 1e-10)
    else:
        print(f"Warning: Volume column '{volume_col}' not found for CMF calculation for {symbol}.")
        df['mfv'] = np.nan
        df['cmf'] = np.nan

    # Calcular Stochastic Oscillator: Indicador de momentum que compara el precio de cierre particular con un rango de sus precios durante un período de tiempo determinado.
    stoch_window = 14
    stoch_smooth_k = 3
    stoch_smooth_d = 3

    # Ensure High and Low columns exist for Stochastic calculation
    if high_col in df.columns and low_col in df.columns:
        # Calculate Lowest Low (LL) and Highest High (HH) over the window
        df['LL'] = df[low_col].rolling(window=stoch_window).min()
        df['HH'] = df[high_col].rolling(window=stoch_window).max()

        # Calculate %K
        # Avoid division by zero if HH equals LL
        df['%K'] = ((df[close_col] - df['LL']) / (df['HH'] - df['LL'] + 1e-10)) * 100

        # Calculate %D (Simple Moving Average of %K)
        df['%D'] = df['%K'].rolling(window=stoch_smooth_d).mean()

        # Smooth %K (optional, but common)
        df['%K_smooth'] = df['%K'].rolling(window=stoch_smooth_k).mean()

        # Drop temporary columns
        df = df.drop(columns=['LL', 'HH'], errors='ignore')
    else:
        print(f"Warning: High or Low columns not found for Stochastic Oscillator calculation for {symbol}.")
        df['%K'] = np.nan
        df['%D'] = np.nan
        df['%K_smooth'] = np.nan


    # Clean up temporary columns at the very end
    # Use a list comprehension to ensure only existing columns are dropped
    cols_to_drop = ['rolling_mean', 'rolling_std', 'tr1', 'tr2', 'tr3', 'tr', 'mf_multiplier', 'mfv']
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns], errors='ignore')


    df = df.dropna() # Eliminar filas con valores NaN generados por los calculos de ventanas moviles y EMA
    return df

def detect_rules(df, z_threshold):
    """
    Aplica reglas simples basadas en umbrales de z-score.
    """
    # Ensure z_price and vol_z exist before accessing
    if 'z_price' not in df.columns or 'vol_z' not in df.columns:
        print("Error: Columnas 'z_price' o 'vol_z' faltantes para aplicar reglas.")
        df["rule_price_z"] = False
        df["rule_vol_spike"] = False
        df["rule_combo"] = False
        return df

    # Indicadores basados en reglas: Identifica puntos donde el Z-score del precio o volumen supera un umbral definido.
    df["rule_price_z"] = df["z_price"].abs() > z_threshold
    df["rule_vol_spike"] = df["vol_z"] > z_threshold # Solo consideramos picos de volumen (vol_z positivo)
    # Combinacion de reglas (ambas condiciones se cumplen): Identifica puntos que cumplen ambas reglas simultaneamente.
    df["rule_combo"] = df["rule_price_z"] & df["rule_vol_spike"]
    return df

def fit_isolationforest(df, features, contamination):
    """
    Entrena y aplica el modelo Isolation Forest para la deteccion de anomalias.
    Isolation Forest aisla las anomalias en lugar de perfilar los puntos normales.
    """
    # Aseguramos que solo usamos las features definidas y manejamos NaNs (rellenando con 0, considerar otras estrategias si es necesario)
    # Filter features to only include those present in the DataFrame columns and are strings
    available_features = [f for f in features if isinstance(f, str) and f in df.columns]
    if not available_features:
        print("Error: No available features (o nombres de features invalidos) para Isolation Forest training.")
        df["if_score"] = np.nan
        df["if_score_norm"] = np.nan
        return df, None # Return df and None model if no features

    X = df[available_features].fillna(0).values
    # Ensure X is not empty after filtering features and handling NaNs
    if X.size == 0 or X.shape[0] == 0:
        print("Error: Features matrix is empty after filtering. Cannot train Isolation Forest.")
        df["if_score"] = np.nan
        df["if_score_norm"] = np.nan
        return df, None

    try:
        model = IsolationForest(n_estimators=200, contamination=contamination, random_state=42)
        model.fit(X)
        # Puntuacion de anomalia (mayor puntuacion -> mas anomalo)
        # Los valores mas bajos de decision_function indican mas anomalias. Se invierte para que un valor mas alto signifique mas anomalia.
        scores = -model.decision_function(X)
        df["if_score"] = scores
        # Normalizar la puntuacion entre 0 y 1 para combinarla con otras metricas mas facilmente.
        # Avoid division by zero if max and min are the same
        score_min = df["if_score"].min()
        score_max = df["if_score"].max()
        score_range = score_max - score_min

        df["if_score_norm"] = (df["if_score"] - score_min) / (score_range + 1e-9) if score_range != 0 else 0
        return df, model
    except Exception as e:
        print(f"Error durante el entrenamiento/aplicacion de Isolation Forest: {e}")
        df["if_score"] = np.nan
        df["if_score_norm"] = np.nan
        return df, None # Return df and None model in case of error


def combined_score(df):
    """
    Calcula un score combinado a partir de los diferentes metodos de deteccion (Isolation Forest, reglas).
    """
    # Ensure z_price and vol_z exist before accessing
    if 'z_price' not in df.columns or 'vol_z' not in df.columns:
        print("Error: Columnas 'z_price' o 'vol_z' faltantes para calcular el score combinado.")
        df["final_score"] = np.nan
        return df


    # Ensemble simple ponderado: Combina la puntuacion normalizada de IF con las puntuaciones normalizadas de los indicadores de reglas.
    # Normalizamos las caracteristicas usadas en las reglas para combinarlas
    z = df["z_price"].abs() # Valor absoluto del Z-score del precio
    zv = df["vol_z"].clip(lower=0)  # Solo consideramos picos de volumen positivos

    # Evitamos division por cero si el rango es 0
    z_min, z_max = z.min(), z.max()
    zv_min, zv_max = zv.min(), zv.max()

    # Ensure z_max and zv_max are not equal to z_min and zv_min respectively to avoid division by zero
    z_range = z_max - z_min
    zv_range = zv_max - zv_min

    df["z_norm"] = (z - z_min) / (z_range + 1e-9) if z_range != 0 else 0 # Normalizacion del Z-score del precio, handle zero range
    df["vol_norm"] = (zv - zv_min) / (zv_range + 1e-9) if zv_range != 0 else 0 # Normalizacion del Z-score del volumen normalizado (solo picos), handle zero range


    # Ponderacion de los scores: Asigna pesos a cada componente del score combinado.
    # Los pesos pueden ser ajustados para dar mas importancia a un metodo u otro.
    # Ensure if_score_norm is not None before using
    if 'if_score_norm' in df.columns and pd.notna(df['if_score_norm']).any():
         df["final_score"] = (0.5 * df["if_score_norm"] + # 50% del score de Isolation Forest
                              0.3 * df["z_norm"] +        # 30% del score Z-score del precio normalizado
                              0.2 * df["vol_norm"])       # 20% del score Z-score del volumen normalizado
    else:
         print("Advertencia: if_score_norm no disponible o contiene solo NaNs. Usando score combinado basado solo en reglas.")
         df["final_score"] = (0.3 * df["z_norm"] +
                              0.2 * df["vol_norm"]) # Fallback score if IF failed or has no valid scores


    return df

def generate_signals(df, signal_anomaly_threshold, symbol, trend_window=50, rsi_buy_threshold=30, rsi_sell_threshold=70):
    """
    Genera señales de compra/venta mas sofisticadas.
    Considera:
    - Score de anomalia combinado.
    - Direccion de la tendencia (basado en una media movil).
    - Valores de indicadores tecnicos especificos (ej. RSI).
    Añade columnas 'buy_signal' y 'sell_signal' al dataframe.
    """
    df = df.copy()

    # Initialize signal columns to False
    df['buy_signal'] = False
    df['sell_signal'] = False

    # Determine the correct 'Close' column name - should be 'Close' after flattening
    close_col_name = "Close"

    if close_col_name not in df.columns:
         print(f"Advertencia: No se encontró la columna '{close_col_name}' para generar señales de trading para {symbol}. Columnas disponibles: {list(df.columns)}")
         return df # Return dataframe without signals

    # --- Sophisticated Signal Logic ---

    # 1. Trend Analysis (using a Simple Moving Average)
    # Calculate the moving average
    df['trend_ma'] = df[close_col_name].rolling(window=trend_window).mean()

    # Determine trend direction (price above MA = uptrend, price below MA = downtrend)
    df['is_uptrend'] = df[close_col_name] > df['trend_ma']
    df['is_downtrend'] = df[close_col_name] < df['trend_ma']

    # 2. Combine Anomaly Score, Trend, and Indicator Conditions (e.g., RSI)
    # Ensure 'final_score' and 'rsi' columns exist
    if 'final_score' in df.columns and pd.notna(df['final_score']).any() and 'rsi' in df.columns and pd.notna(df['rsi']).any():

        # Buy signal: High anomaly score AND (is_uptrend OR price is starting to move up) AND (RSI is oversold OR moving upwards from oversold)
        # A simplified example: High anomaly score AND Uptrend AND RSI is below a threshold
        df.loc[
            (df['final_score'] > signal_anomaly_threshold) &
            (df['is_uptrend'] == True) &
            (df['rsi'] < rsi_buy_threshold),
            'buy_signal'
        ] = True

        # Sell signal: High anomaly score AND (is_downtrend OR price is starting to move down) AND (RSI is overbought OR moving downwards from overbought)
        # A simplified example: High anomaly score AND Downtrend AND RSI is above a threshold
        df.loc[
            (df['final_score'] > signal_anomaly_threshold) &
            (df['is_downtrend'] == True) &
            (df['rsi'] > rsi_sell_threshold),
            'sell_signal'
        ] = True

    else:
         print("Advertencia: Columna 'final_score' o 'rsi' no disponible/contiene NaNs. No se pueden generar señales basadas en el score de anomalia y RSI.")


    # Clean up temporary columns used for signaling logic if desired
    df = df.drop(columns=['trend_ma', 'is_uptrend', 'is_downtrend'], errors='ignore')


    # Note: This is a basic example. More complex logic could involve:
    # - Using different types of moving averages (EMA, etc.)
    # - Considering crossovers of MAs
    # - Incorporating other indicators (MACD crossovers, volume surges not captured by vol_z, etc.)
    # - Implementing entry/exit logic (e.g., buy signal starts a potential trade, sell signal exits it)

    return df # Return dataframe with added signal columns

# --- Backtesting Module ---

def backtest_strategy(df, initial_capital=10000, commission_rate=0.001):
    """
    Simula la ejecucion de una estrategia de trading basada en señales de compra/venta.
    Calcula metricas de rendimiento como P/L, numero de operaciones, tasa de exito, etc.

    Args:
        df (pd.DataFrame): DataFrame con datos de precios y señales ('Close', 'buy_signal', 'sell_signal').
        initial_capital (float): Capital inicial para la simulacion.
        commission_rate (float): Tasa de comision por operacion (ej. 0.001 para 0.1%).

    Returns:
        dict: Un diccionario que contiene las metricas de backtesting.
    """
    if df is None or df.empty:
        print("Error: DataFrame vacio o nulo proporcionado para backtesting.")
        return None

    if 'Close' not in df.columns or 'buy_signal' not in df.columns or 'sell_signal' not in df.columns:
        print("Error: DataFrame no contiene las columnas necesarias ('Close', 'buy_signal', 'sell_signal') para backtesting.")
        return None

    # Initialize variables for backtesting
    capital = initial_capital
    position = 0  # 0: No position, 1: Long position
    trades = []
    entry_price = 0
    peak_capital = initial_capital
    max_drawdown = 0

    # Iterate through the dataframe to simulate trades
    for i in range(len(df)):
        # Get current data point
        current_price = df['Close'].iloc[i]
        buy_signal = df['buy_signal'].iloc[i]
        sell_signal = df['sell_signal'].iloc[i]
        current_date = df.index[i]

        # --- Trading Logic ---

        # Check for Buy Signal and if not already in a position
        if buy_signal and position == 0:
            # Calculate maximum shares to buy with available capital
            # Consider commission: price * shares * (1 + commission_rate) <= capital
            # shares <= capital / (price * (1 + commission_rate))
            max_shares = capital / (current_price * (1 + commission_rate))
            shares_to_buy = int(max_shares) # Buy whole shares

            if shares_to_buy > 0:
                 cost = shares_to_buy * current_price * (1 + commission_rate)
                 capital -= cost
                 position = shares_to_buy
                 entry_price = current_price
                 print(f"{current_date.strftime('%Y-%m-%d %H:%M:%S')} - BUY: {shares_to_buy} shares at {current_price:.2f}. Capital remaining: {capital:.2f}")


        # Check for Sell Signal and if in a long position
        elif sell_signal and position > 0:
            # Calculate proceeds from selling
            proceeds = position * current_price * (1 - commission_rate)
            profit_loss = proceeds - (position * entry_price * (1 + commission_rate)) # Calculate P/L based on entry cost including commission
            capital += proceeds

            trades.append({
                'entry_date': df.index[df['Close'] == entry_price].max(), # Find date of entry price (might not be exact match, find closest or use index)
                'exit_date': current_date,
                'entry_price': entry_price,
                'exit_price': current_price,
                'shares': position,
                'profit_loss': profit_loss,
                'trade_type': 'Long' # Only long trades in this simple strategy
            })

            print(f"{current_date.strftime('%Y-%m-%d %H:%M:%S')} - SELL: {position} shares at {current_price:.2f}. P/L: {profit_loss:.2f}. Capital: {capital:.2f}")

            # Reset position
            position = 0
            entry_price = 0

        # --- Capital Tracking and Drawdown Calculation ---
        current_portfolio_value = capital + (position * current_price)
        peak_capital = max(peak_capital, current_portfolio_value)
        drawdown = (peak_capital - current_portfolio_value) / peak_capital if peak_capital > 0 else 0
        max_drawdown = max(max_drawdown, drawdown)


    # Close any open position at the end of the data
    if position > 0:
        current_price = df['Close'].iloc[-1]
        current_date = df.index[-1]
        proceeds = position * current_price * (1 - commission_rate)
        profit_loss = proceeds - (position * entry_price * (1 + commission_rate))
        capital += proceeds

        trades.append({
            'entry_date': df.index[df['Close'] == entry_price].max(),
            'exit_date': current_date,
            'entry_price': entry_price,
            'exit_price': current_price,
            'shares': position,
            'profit_loss': profit_loss,
            'trade_type': 'Long'
        })
        print(f"{current_date.strftime('%Y-%m-%d %H:%M:%S')} - FINAL SELL: {position} shares at {current_price:.2f}. P/L: {profit_loss:.2f}. Final Capital: {capital:.2f}")


    # --- Calculate Backtesting Metrics ---
    total_profit_loss = capital - initial_capital
    num_trades = len(trades)
    winning_trades = [trade for trade in trades if trade['profit_loss'] > 0]
    losing_trades = [trade for trade in trades if trade['profit_loss'] <= 0] # Includes zero profit trades as non-winners

    win_rate = len(winning_trades) / num_trades if num_trades > 0 else 0
    total_winning_profit = sum(trade['profit_loss'] for trade in winning_trades)
    total_losing_loss = sum(trade['profit_loss'] for trade in losing_trades)

    avg_profit_per_winning_trade = total_winning_profit / len(winning_trades) if len(winning_trades) > 0 else 0
    avg_loss_per_losing_trade = total_losing_loss / len(losing_trades) if len(losing_trades) > 0 else 0 # This will be negative or zero

    # Profit Factor: Total Gross Profit / Total Gross Loss (Absolute value)
    # Avoid division by zero if there are no losing trades
    profit_factor = total_winning_profit / abs(total_losing_loss) if total_losing_loss < 0 else (total_winning_profit if total_losing_loss == 0 else np.nan) # Handle cases with no losses or zero losses


    backtest_results = {
        'initial_capital': initial_capital,
        'final_capital': capital,
        'total_profit_loss': total_profit_loss,
        'return_percentage': (total_profit_loss / initial_capital) * 100 if initial_capital > 0 else 0,
        'num_trades': num_trades,
        'win_rate': win_rate,
        'total_winning_profit': total_winning_profit,
        'total_losing_loss': total_losing_loss,
        'avg_profit_per_winning_trade': avg_profit_per_winning_trade,
        'avg_loss_per_losing_trade': avg_loss_per_losing_trade,
        'profit_factor': profit_factor,
        'max_drawdown': max_drawdown,
        'trades': trades # Optionally include the list of individual trades
    }

    return backtest_results


def run_anomaly_detection(symbol, period, interval, z_threshold, if_contamination, if_features, anomaly_threshold, trend_window=50, rsi_buy_threshold=30, rsi_sell_threshold=70):
    """
    Ejecuta el proceso completo de deteccion de anomalias para un simbolo.
    Retorna el dataframe procesado con resultados de anomalias y señales.
    (No envia mensaje de Discord aqui, eso se hace en el main interactivo)
    """
    print(f"\n--- Procesando simbolo: {symbol} ---")
    df = fetch_data(symbol, period, interval)
    if df is None or df.empty: # Check for None or empty df from fetch_data
        print(f"Saltando procesamiento para {symbol} debido a falta de datos.")
        return None # Return None if no data or error in fetch_data

    df = feature_engineer(df, symbol) # Pass symbol to feature_engineer
    if df.empty: # Check if feature engineering returned empty
        print(f"Saltando procesamiento para {symbol} debido a falta de datos despues de feature engineering.")
        return None

    df = detect_rules(df, z_threshold)
    # Aseguramos que el dataframe no esta vacio despues del feature engineering
    if df.empty:
         print(f"Saltando procesamiento para {symbol} debido a falta de datos despues de deteccion por reglas.")
         return None

    df, if_model = fit_isolationforest(df, if_features, if_contamination)
    if df is None or df.empty: # Check if Isolation Forest returned empty or None model
        print(f"Saltando procesamiento para {symbol} debido a error o falta de datos despues de Isolation Forest.")
        return None

    df = combined_score(df)
    if df.empty: # Check if combined scoring returned empty
        print(f"Saltando procesamiento para {symbol} debido a falta de datos despues de calculo de score combinado.")
        return None

    # Add signal generation logic - Pass the new parameters
    df = generate_signals(df, anomaly_threshold, symbol, trend_window, rsi_buy_threshold, rsi_sell_threshold) # Use the main ANOMALY_THRESHOLD for signals and pass symbol, plus new params
    if df.empty: # Check if signal generation returned empty
        print(f"Saltando procesamiento para {symbol} debido a falta de datos despues de generacion de señales.")
        return None

    # Obtenemos el ultimo punto de datos para el resumen
    # Ensure dataframe is not empty before accessing iloc[-1]
    if df.empty:
        latest = None
    else:
         latest = df.iloc[-1]


    # Summarize results for the latest point
    if latest is not None:
        score = latest.get("final_score", pd.Series([np.nan])).item()
        rule_combo_status = bool(latest.get("rule_combo", False))
        buy_signal_latest = bool(latest.get('buy_signal', False))
        sell_signal_latest = bool(latest.get('sell_signal', False))

        print(f"\nResumen para el ultimo punto de {symbol}:")
        print(f"Score Combinado: {score:.3f}" if pd.notna(score) else "Score Combinado: N/A")
        print(f"Regla combinada activa: {rule_combo_status}")
        print(f"Señal de compra: {buy_signal_latest}")
        print(f"Señal de venta: {sell_signal_latest}")

        # Check for alert condition for the last point processed
        if (pd.notna(score) and score > anomaly_threshold) or rule_combo_status or buy_signal_latest or sell_signal_latest:
             print(f"\n¡ALERTA detectada para {symbol} en el último punto procesado!")
        else:
             print(f"\nNo hay ALERTA para {symbol} en el último punto procesado.")


    return df # Return the processed dataframe

# --- Main Execution with Interactive Menu ---
if __name__ == "__main__":
    print("--- Bot de Detección de Anomalías y Señales ---")

    # Declare global variables first
    global SYMBOLS
    global PERIOD
    global INTERVAL
    global Z_THRESHOLD
    global IF_CONTAMINATION
    global IF_FEATURES
    global DISCORD_WEBHOOK_URL
    global ANOMALY_THRESHOLD
    global TREND_WINDOW
    global RSI_BUY_THRESHOLD
    global RSI_SELL_THRESHOLD
    global INITIAL_CAPITAL
    global COMMISSION_RATE


    # New: Ask user if they want to load configuration from a file
    load_config_choice = input("¿Desea cargar la configuracion desde un archivo (ej. config.json)? (s/n): ").lower()
    config_loaded = False

    # Define variables with defaults (moved assignment after global declaration)
    SYMBOLS = ["BTC-USD", "ETH-USD"]
    PERIOD = "30d"
    INTERVAL = "5m"
    Z_THRESHOLD = 3.0
    IF_CONTAMINATION = 0.01
    IF_FEATURES = ["logret", "vol_20", "z_price", "vol_z", "price_vol_interaction", "rsi", "macd", "upper_band", "lower_band", "atr", "cmf", "%K_smooth", "%D"]
    DISCORD_WEBHOOK_URL = ""
    ANOMALY_THRESHOLD = 0.6
    TREND_WINDOW = 50
    RSI_BUY_THRESHOLD = 30
    RSI_SELL_THRESHOLD = 70
    INITIAL_CAPITAL = 10000
    COMMISSION_RATE = 0.001


    if load_config_choice == 's':
        config_file_path = input("Ingrese la ruta del archivo de configuracion (dejar en blanco para 'config.json'): ") or "config.json"
        try:
            print(f"Intentando cargar configuracion desde '{config_file_path}'...")
            with open(config_file_path, 'r') as f:
                config = json.load(f)

            # Update global configuration variables with values from the loaded dictionary
            # Use .get() with default values to handle missing keys gracefully
            # Ensure types are correct (e.g., float for thresholds, list for symbols/features)

            SYMBOLS = [s.strip() for s in config.get("SYMBOLS", "").split(',') if s.strip()] if isinstance(config.get("SYMBOLS"), str) else SYMBOLS


            PERIOD = config.get("PERIOD", PERIOD)


            INTERVAL = config.get("INTERVAL", INTERVAL)


            Z_THRESHOLD = float(config.get("Z_THRESHOLD", Z_THRESHOLD)) if isinstance(config.get("Z_THRESHOLD"), (int, float, str)) else Z_THRESHOLD


            IF_CONTAMINATION = float(config.get("IF_CONTAMINATION", IF_CONTAMINATION)) if isinstance(config.get("IF_CONTAMINATION"), (int, float, str)) else IF_CONTAMINATION


            # Expect IF_FEATURES to be a list in JSON, fallback to default string if not list
            if isinstance(config.get("IF_FEATURES"), list):
                IF_FEATURES = [f.strip() for f in config.get("IF_FEATURES") if isinstance(f, str) and f.strip()]
            elif isinstance(config.get("IF_FEATURES"), str):
                IF_FEATURES = [f.strip() for f in config.get("IF_FEATURES").split(',') if f.strip()]
            else:
                 IF_FEATURES = IF_FEATURES


            DISCORD_WEBHOOK_URL = config.get("DISCORD_WEBHOOK_URL", DISCORD_WEBHOOK_URL)


            ANOMALY_THRESHOLD = float(config.get("ANOMALY_THRESHOLD", ANOMALY_THRESHOLD)) if isinstance(config.get("ANOMALY_THRESHOLD"), (int, float, str)) else ANOMALY_THRESHOLD

            # New: Load signal parameters from config

            TREND_WINDOW = int(config.get("TREND_WINDOW", TREND_WINDOW)) if isinstance(config.get("TREND_WINDOW"), (int, str)) else TREND_WINDOW


            RSI_BUY_THRESHOLD = float(config.get("RSI_BUY_THRESHOLD", RSI_BUY_THRESHOLD)) if isinstance(config.get("RSI_BUY_THRESHOLD"), (int, float, str)) else RSI_BUY_THRESHOLD


            RSI_SELL_THRESHOLD = float(config.get("RSI_SELL_THRESHOLD", RSI_SELL_THRESHOLD)) if isinstance(config.get("RSI_SELL_THRESHOLD"), (int, float, str)) else RSI_SELL_THRESHOLD

            # New: Load backtesting parameters from config
            INITIAL_CAPITAL = float(config.get("INITIAL_CAPITAL", INITIAL_CAPITAL)) if isinstance(config.get("INITIAL_CAPITAL"), (int, float, str)) else INITIAL_CAPITAL
            COMMISSION_RATE = float(config.get("COMMISSION_RATE", COMMISSION_RATE)) if isinstance(config.get("COMMISSION_RATE"), (int, float, str)) else COMMISSION_RATE


            config_loaded = True
            print(f"Configuracion cargada exitosamente desde '{config_file_path}'.")

        except FileNotFoundError:
            print(f"Error: Archivo de configuracion '{config_file_path}' no encontrado. Procediendo con entrada manual.")
        except json.JSONDecodeError:
            print(f"Error: No se pudo decodificar el JSON del archivo '{config_file_path}'. Asegurese de que el archivo esta formateado correctamente. Procediendo con entrada manual.")
        except Exception as e:
             print(f"Ocurrio un error inesperado al cargar la configuracion desde '{config_file_path}': {e}. Procediendo con entrada manual.")


    # If configuration was not loaded from file, or if loading failed, prompt for manual input
    if not config_loaded:
        print("Por favor, ingresa los parámetros de configuración manualmente.")
        # Get configuration inputs from the user (existing code)
        symbols_input = input("Ingrese los simbolos a monitorear separados por coma (ej. BTC-USD,ETH-USD): ")
        # Use global SYMBOLS list here as it's referenced in fetch_data
        SYMBOLS = [s.strip() for s in symbols_input.split(',') if s.strip()] if symbols_input else SYMBOLS

        # Exit if no symbols are provided
        if not SYMBOLS:
            print("No se ingresaron simbolos. Terminando ejecucion.")
            processed_dfs = {} # Ensure processed_dfs is defined even if symbols are empty
        else:

            period_input = input(f"Ingrese el periodo de descarga de datos para {SYMBOLS} (ej. 30d, 1y): ") or PERIOD
            PERIOD = period_input

            interval_input = input(f"Ingrese el intervalo de los datos para {SYMBOLS} (ej. 5m, 1h, 1d): ") or INTERVAL
            INTERVAL = interval_input

            z_threshold_input = input(f"Ingrese el umbral para Z-score (ej. {Z_THRESHOLD}): ") or str(Z_THRESHOLD)
            Z_THRESHOLD = float(z_threshold_input)

            if_contamination_input = input(f"Ingrese la fraccion de anomalias esperadas para Isolation Forest (entre 0 y 1, ej. {IF_CONTAMINATION}): ") or str(IF_CONTAMINATION)
            IF_CONTAMINATION = float(if_contamination_input)

            if_features_input = input(f"Ingrese las features para Isolation Forest separadas por coma (ej. logret,vol_20,z_price,rsi,macd,upper_band,lower_band,atr,cmf): ")
            IF_FEATURES = [f.strip() for f in if_features_input.split(',') if f.strip()] if if_features_input else IF_FEATURES

            discord_webhook_url_input = input("Ingrese la URL del Webhook de Discord (dejar en blanco para no usar Discord): ")
            DISCORD_WEBHOOK_URL = discord_webhook_url_input

            anomaly_threshold_input = input(f"Ingrese el umbral de score combinado para enviar alerta (ej. {ANOMALY_THRESHOLD}): ") or str(ANOMALY_THRESHOLD)
            ANOMALY_THRESHOLD = float(anomaly_threshold_input)

            # New: Prompt for signal parameters if not loaded
            trend_window_input = input(f"Ingrese la ventana de la media movil para la tendencia (ej. {TREND_WINDOW}): ") or str(TREND_WINDOW)
            TREND_WINDOW = int(trend_window_input)

            rsi_buy_threshold_input = input(f"Ingrese el umbral de RSI para señal de COMPRA (ej. {RSI_BUY_THRESHOLD}): ") or str(RSI_BUY_THRESHOLD)
            RSI_BUY_THRESHOLD = float(rsi_buy_threshold_input)

            rsi_sell_threshold_input = input(f"Ingrese el umbral de RSI para señal de VENTA (ej. {RSI_SELL_THRESHOLD}): ") or str(RSI_SELL_THRESHOLD)
            RSI_SELL_THRESHOLD = float(rsi_sell_threshold_input)

            # New: Prompt for backtesting parameters if not loaded
            initial_capital_input = input(f"Ingrese el capital inicial para backtesting (ej. {INITIAL_CAPITAL}): ") or str(INITIAL_CAPITAL)
            INITIAL_CAPITAL = float(initial_capital_input)

            commission_rate_input = input(f"Ingrese la tasa de comision por operacion (ej. {COMMISSION_RATE} para 0.1%): ") or str(COMMISSION_RATE)
            COMMISSION_RATE = float(commission_rate_input)


    # Continue with the rest of the execution only if symbols are available (either loaded or manually entered)
    if SYMBOLS:
        print("\nIniciando el proceso de procesamiento de datos...")
        processed_dfs = {} # Dictionary to store processed dataframes for each symbol
        for sym in SYMBOLS:
            # Pass all configuration parameters, including new signal and backtesting ones
            processed_dfs[sym] = run_anomaly_detection(sym, PERIOD, INTERVAL, Z_THRESHOLD, IF_CONTAMINATION, IF_FEATURES, ANOMALY_THRESHOLD, TREND_WINDOW, RSI_BUY_THRESHOLD, RSI_SELL_THRESHOLD)


        print("\nProceso de procesamiento de datos finalizado.")

        # --- Interactive Menu Options ---
        print("\n--- Opciones ---")

        # 1. Display Final Processed DataFrame
        display_df_choice = input("¿Desea mostrar el dataframe final procesado para el primer simbolo? (s/n): ").lower()
        if display_df_choice == 's':
            if SYMBOLS and SYMBOLS[0] in processed_dfs and processed_dfs[SYMBOLS[0]] is not None and not processed_dfs[SYMBOLS[0]].empty:
                df_final = processed_dfs[SYMBOLS[0]]
                print(f"\nFinal processed DataFrame for {SYMBOLS[0]}:")
                display(df_final.tail()) # Use display for DataFrames in Colab
            elif SYMBOLS:
                print(f"\nNo se pudo obtener o procesar datos para el primer simbolo: {SYMBOLS[0]}. No se puede mostrar el dataframe.")
            else:
                 print("\nNo se especificaron simbolos para procesar.")


        # 2. Visualize Data and Anomalies (for the first symbol processed)
        visualize_choice = input(f"¿Desea ver los gráficos de datos, características, anomalías y señales para {SYMBOLS[0]}? (s/n): ").lower()
        if visualize_choice == 's':
            if SYMBOLS and SYMBOLS[0] in processed_dfs and processed_dfs[SYMBOLS[0]] is not None and not processed_dfs[SYMBOLS[0]].empty:
                df_to_visualize = processed_dfs[SYMBOLS[0]]
                visualize_data_features(df_to_visualize, SYMBOLS[0], SYMBOLS)
                visualize_anomalies(df_to_visualize, SYMBOLS[0], ANOMALY_THRESHOLD)
            elif SYMBOLS:
                print(f"\nNo se pudo obtener o procesar datos para el primer simbolo: {SYMBOLS[0]}. No se pueden generar gráficos.")
            else:
                 print("\nNo se especificaron simbolos para procesar.")

        # 3. Run Backtesting (for the first symbol processed)
        backtest_choice = input(f"¿Desea ejecutar el backtesting para el primer simbolo ({SYMBOLS[0]})? (s/n): ").lower() if SYMBOLS else 'n'
        if backtest_choice == 's':
             if SYMBOLS and SYMBOLS[0] in processed_dfs and processed_dfs[SYMBOLS[0]] is not None and not processed_dfs[SYMBOLS[0]].empty:
                  df_to_backtest = processed_dfs[SYMBOLS[0]]
                  print(f"\nEjecutando backtesting para {SYMBOLS[0]} con Capital Inicial: {INITIAL_CAPITAL}, Comisión: {COMMISSION_RATE}...")
                  backtest_results = backtest_strategy(df_to_backtest, initial_capital=INITIAL_CAPITAL, commission_rate=COMMISSION_RATE)

                  if backtest_results:
                       print("\n--- Resultados del Backtesting ---")
                       print(f"Símbolo: {SYMBOLS[0]}")
                       print(f"Capital Inicial: {backtest_results['initial_capital']:.2f}")
                       print(f"Capital Final: {backtest_results['final_capital']:.2f}")
                       print(f"P/L Total: {backtest_results['total_profit_loss']:.2f}")
                       print(f"Retorno Porcentual: {backtest_results['return_percentage']:.2f}%")
                       print(f"Número de Operaciones: {backtest_results['num_trades']}")
                       print(f"Tasa de Éxito: {backtest_results['win_rate']:.2f}")
                       print(f"Ganancia Total (Operaciones Ganadoras): {backtest_results['total_winning_profit']:.2f}")
                       print(f"Pérdida Total (Operaciones Perdedoras): {backtest_results['total_losing_loss']:.2f}")
                       print(f"Ganancia Promedio por Operación Ganadora: {backtest_results['avg_profit_per_winning_trade']:.2f}")
                       print(f"Pérdida Promedio por Operación Perdedora: {backtest_results['avg_loss_per_losing_trade']:.2f}")
                       print(f"Factor de Ganancia: {backtest_results['profit_factor']:.2f}" if pd.notna(backtest_results['profit_factor']) else "Factor de Ganancia: N/A")
                       print(f"Máximo Drawdown: {backtest_results['max_drawdown']:.2f}")

                       # Optionally display individual trades
                       display_trades_choice = input("¿Desea mostrar los detalles de cada operación? (s/n): ").lower()
                       if display_trades_choice == 's' and 'trades' in backtest_results and backtest_results['trades']:
                            print("\n--- Detalles de las Operaciones ---")
                            trades_df = pd.DataFrame(backtest_results['trades'])
                            display(trades_df)
                       elif display_trades_choice == 's':
                            print("No hay operaciones para mostrar.")

                  else:
                       print("\nNo se pudieron calcular los resultados del backtesting.")

             elif SYMBOLS:
                 print(f"\nNo se pudo obtener o procesar datos para el primer simbolo: {SYMBOLS[0]}. No se puede ejecutar el backtesting.")
             else:
                 print("\nNo se especificaron simbolos para procesar.")


        # 4. Send Discord Alert (based on the last point processed for EACH symbol)
        # Shifted Discord alert to be option 4 after backtesting
        print("\n--- Verificación de Alertas (último punto procesado por cada simbolo) ---")
        for sym in SYMBOLS:
            if sym in processed_dfs and processed_dfs[sym] is not None and not processed_dfs[sym].empty:
                 df_sym = processed_dfs[sym]
                 latest_sym = df_sym.iloc[-1]

                 score_sym = latest_sym.get("final_score", pd.Series([np.nan])).item()
                 rule_combo_status_sym = bool(latest_sym.get("rule_combo", False))
                 buy_signal_latest_sym = bool(latest_sym.get('buy_signal', False))
                 sell_signal_latest_sym = bool(latest_sym.get('sell_signal', False))

                 # Construct message if alert condition is met for the latest point of THIS symbol
                 # Updated alert message to include more details about the signal conditions if a signal is generated
                 if (pd.notna(score_sym) and score_sym > ANOMALY_THRESHOLD) or rule_combo_status_sym or buy_signal_latest_sym or sell_signal_latest_sym:
                     msg_lines = [f"🚨 ALERTA {sym} (Último Punto Procesado) 🚨"]
                     latest_time_formatted = "N/A"
                     try:
                         latest_time_str = str(latest_sym.name)
                         try:
                             latest_time_naive = datetime.fromisoformat(latest_time_str.replace('Z', '+00:00'))
                         except ValueError:
                              try:
                                   latest_time_naive = datetime.strptime(latest_time_str.split('.')[0].split('+')[0], '%Y-%m-%d %H:%M:%S')
                              except ValueError:
                                   latest_time_naive = None


                         if latest_time_naive:
                              latest_time_formatted = latest_time_naive.strftime('%Y-%m-%d %H:%M:%S') + " UTC"
                         else:
                              latest_time_formatted = str(latest_sym.name)

                     except Exception as e:
                         print(f"Error formatting timestamp for {sym}: {e}")
                         latest_time_formatted = str(latest_sym.name)


                     msg_lines.append(f"Time: {latest_time_formatted}")
                     msg_lines.append(f"Score Combinado: {score_sym:.3f}" if pd.notna(score_sym) else "Score Combinado: N/A")
                     close_price_latest_sym = latest_sym.get("Close", np.nan)
                     z_price_latest_sym = latest_sym.get("z_price", np.nan)
                     vol_z_latest_sym = latest_sym.get("vol_z", np.nan)

                     msg_lines.append(f"Precio: {close_price_latest_sym:.2f}" if pd.notna(close_price_latest_sym) else "Precio: N/A")
                     msg_lines.append(f"z_precio: {z_price_latest_sym:.2f}" if pd.notna(z_price_latest_sym) else "z_precio: N/A")
                     msg_lines.append(f"vol_z: {vol_z_latest_sym:.2f}" if pd.notna(vol_z_latest_sym) else "vol_z: N/A")

                     msg_lines.append(f"Regla combinada activa: {rule_combo_status_sym}")

                     # Add signal information to the message if they exist and are True
                     if 'buy_signal' in latest_sym and latest_sym['buy_signal'].item():
                         msg_lines.append("📊 SEÑAL: COMPRA")
                         # Add details if available
                         if 'is_uptrend' in latest_sym: msg_lines.append(f"  - Tendencia: {'Alcista' if latest_sym['is_uptrend'].item() else 'Bajista/Lateral'}")
                         if 'rsi' in latest_sym: msg_lines.append(f"  - RSI: {latest_sym['rsi'].item():.2f} (Umbral Compra: {RSI_BUY_THRESHOLD})")
                         if 'final_score' in latest_sym: msg_lines.append(f"  - Score Anomalia: {latest_sym['final_score'].item():.3f}")


                     if 'sell_signal' in latest_sym and latest_sym['sell_signal'].item():
                         msg_lines.append("📉 SEÑAL: VENTA")
                         # Add details if available
                         if 'is_downtrend' in latest_sym: msg_lines.append(f"  - Tendencia: {'Bajista' if latest_sym['is_downtrend'].item() else 'Alcista/Lateral'}")
                         if 'rsi' in latest_sym: msg_lines.append(f"  - RSI: {latest_sym['rsi'].item():.2f} (Umbral Venta: {RSI_SELL_THRESHOLD})")
                         if 'final_score' in latest_sym: msg_lines.append(f"  - Score Anomalia: {latest_sym['final_score'].item():.3f}")


                     alert_msg = "\n".join(msg_lines)
                     print("\n" + alert_msg)

                     send_discord_message(alert_msg, DISCORD_WEBHOOK_URL)
                 else:
                     print(f"\n✅ {sym}: OK, no se detecto alerta en el último punto procesado.")
            elif sym in SYMBOLS:
                 print(f"\nNo se pudo procesar datos para {sym}. No se verificó alerta.")


        # 5. Download Processed DataFrame as CSV (for the first symbol processed)
        # Shifted Download to be option 5 after backtesting and alerts
        download_choice = input(f"\n¿Desea descargar el dataframe procesado para el primer simbolo ({SYMBOLS[0]}) como CSV? (s/n): ").lower() if SYMBOLS else 'n'
        if download_choice == 's':
             if SYMBOLS and SYMBOLS[0] in processed_dfs and processed_dfs[SYMBOLS[0]] is not None and not processed_dfs[SYMBOLS[0]].empty:
                 df_to_download = processed_dfs[SYMBOLS[0]]
                 csv_filename = f"{SYMBOLS[0]}_anomalies_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
                 try:
                     df_to_csv_ready = df_to_download.copy()
                     if df_to_csv_ready.index.name:
                         df_to_csv_ready.index.name = 'Date'
                         df_to_csv_ready = df_to_csv_ready.reset_index()
                     else:
                         df_to_csv_ready = df_to_csv_ready.reset_index()
                         if 'index' in df_to_csv_ready.columns:
                              df_to_csv_ready = df_to_csv_ready.rename(columns={'index': 'Date'})


                     df_to_csv_ready.to_csv(csv_filename, index=False)
                     print(f"\nDataFrame guardado como '{csv_filename}'. Iniciando descarga...")
                     try:
                         files.download(csv_filename)
                         print("Descarga iniciada.")
                     except Exception as e:
                          print(f"Error durante la descarga: {e}. Puedes intentar descargar el archivo manualmente desde el explorador de archivos de Colab.")
                 except Exception as e:
                     print(f"Error al preparar o guardar el CSV: {e}")


             elif SYMBOLS:
                 print(f"\nNo se pudo obtener o procesar datos para el primer simbolo: {SYMBOLS[0]}. No se puede descargar el CSV.")
             else:
                 print("\nNo se especificaron simbolos para procesar.")


    print("\n--- Fin de la Ejecución ---")

--- Bot de Detección de Anomalías y Señales ---
¿Desea cargar la configuracion desde un archivo (ej. config.json)? (s/n): n
Por favor, ingresa los parámetros de configuración manualmente.
Ingrese los simbolos a monitorear separados por coma (ej. BTC-USD,ETH-USD): BTC-USD
Ingrese el periodo de descarga de datos para ['BTC-USD'] (ej. 30d, 1y): 1Y
Ingrese el intervalo de los datos para ['BTC-USD'] (ej. 5m, 1h, 1d): 1D
Ingrese el umbral para Z-score (ej. 3.0): 3.0
Ingrese la fraccion de anomalias esperadas para Isolation Forest (entre 0 y 1, ej. 0.01): 0.01
Ingrese las features para Isolation Forest separadas por coma (ej. logret,vol_20,z_price,rsi,macd,upper_band,lower_band,atr,cmf): logret,vol_20,z_price,rsi,macd
Ingrese la URL del Webhook de Discord (dejar en blanco para no usar Discord): 
Ingrese el umbral de score combinado para enviar alerta (ej. 0.6): 0.6
Ingrese la ventana de la media movil para la tendencia (ej. 50): 50
Ingrese el umbral de RSI para señal de COMPRA (ej. 30): 30
In

,Open,High,Low,Close,Volume,logret,vol_10,vol_20,z_price,vol_z,...,rule_price_z,rule_vol_spike,rule_combo,if_score,if_score_norm,z_norm,vol_norm,final_score,buy_signal,sell_signal
Date,,,,,,,,,,,,,,,,,,,,,
2025-09-10,111531.250000,114275.250000,110940.078125,113955.359375,56377473784,0.021508,0.156709,0.277772,1.124917,-0.144489,...,False,False,False,-0.186670,0.188488,0.307850,0.0,0.186599,False,False
2025-09-11,113961.429688,115522.546875,113453.835938,115507.539062,45685065332,0.013529,0.161203,0.244095,2.003160,-0.689093,...,False,False,False,-0.172707,0.232693,0.552318,0.0,0.282042,False,False
2025-09-12,115507.789062,116769.382812,114794.484375,116101.578125,54785725894,0.005130,0.146409,0.240385,2.190588,-0.133692,...,False,False,False,-0.171151,0.237620,0.604491,0.0,0.300157,False,False
2025-09-13,116093.562500,116334.632812,115248.273438,115950.507812,34549454947,-0.001302,0.149043,0.231951,1.893119,-1.228189,...,False,False,False,-0.195119,0.161742,0.521687,0.0,0.237377,False,False
2025-09-15,115377.843750,115554.171875,114914.414062,115443.796875,33916383232,-0.004380,0.139540,0.202716,1.478238,-1.183846,...,False,False,False,-0.188270,0.183425,0.406201,0.0,0.213572,False,False
